In [1]:
!pip install --upgrade pip
!pip install imutils
!pip install opencv-python
# !pip install --upgrade scikit-learn==0.23.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip show scikit-learn

Name: scikit-learn
Version: 1.0.2
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: /usr/local/lib/python3.10/dist-packages
Requires: joblib, numpy, scipy, threadpoolctl
Required-by: fastai, imbalanced-learn, librosa, lightgbm, mlxtend, qudida, sklearn-pandas, yellowbrick


In [3]:
import numpy as np
import cv2
from imutils import paths
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier

Работа с файлами

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


При считывании набора данных (картинок) из папки для построения гистограммы отсортируйте названия по алфавиту, например, так (иначе тренировочные наборы данных могут отличаться даже при фиксированном `random_seed`):

In [5]:
train_image_paths = sorted(list(paths.list_images('/content/drive/My Drive/4_ensemble/train')))

Для работы с изображениями и получения их гистограмм - характеристик распределения интенсивности изображения, следует воспользоваться следующей функцией и библиотекой cv2:

In [6]:
def extract_histogram(image, bins=(8, 8, 8)):
    hist = cv2.calcHist([image], [0, 1, 2], None, bins, [0, 256, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

1. Обучите базовые алгоритмы на исходном наборе данных (предварительно закодируйте классы: Cat - 1, Dog - 0)

In [7]:
X_train = []
y_train = []
for i in train_image_paths:
    X_train.append(extract_histogram(cv2.imread(i)))
    if i.split(sep='.')[0].split(sep='/')[-1] == 'cat':
        y_train.append(1)
    elif i.split(sep='.')[0].split(sep='/')[-1] == 'dog':
        y_train.append(0)
    else:
        print('Error')

- Классификатор с мягким зазором и параметрами: `C = 1.92`, `random_state = 92`, остальные параметры по умолчанию

In [8]:
svm_clf = LinearSVC(C=1.92, random_state=92).fit(X_train, y_train)

- Бэггинг деревьев принятия решений. Параметры дерева: `criterion = 'entropy'`, `min_samples_leaf = 10`, `max_leaf_nodes = 20`, `random_state = 92`, остальные параметры по умолчанию. Параметры бэггинга: `n_estimators = 20`, `random_state = 92`;

In [9]:
bag_clf = BaggingClassifier(DecisionTreeClassifier(criterion='entropy', 
                                                   min_samples_leaf=10, 
                                                   max_leaf_nodes=20,
                                                   random_state=92),
                            n_estimators=20, 
                            random_state=92).fit(X_train, y_train)

- Случайный лес с параметрами: `n_estimators = 20`, `criterion = 'entropy'`, `min_samples_leaf = 10`, `max_leaf_nodes = 20`, `random_state = 92`, остальные параметры по умолчанию.

In [10]:
rfc_clf = RandomForestClassifier(n_estimators=20, 
                                 criterion='entropy', 
                                 min_samples_leaf=10, 
                                 max_leaf_nodes=20, 
                                 random_state=92).fit(X_train, y_train)

2. Инициализируйте метаалгоритм (без обучения) — логистическая регрессия: `solver='lbfgs'`, `random_state = 92`, остальные параметры по умолчанию.

In [11]:
logr_clf = LogisticRegression(solver='lbfgs', random_state=92)

3. Обучите модель стэкинга. Используйте 2-fold (`cv = 2`) кросс-валидацию. Обучение стекинга будет выглядить так:

In [12]:
base_estimators = [('SVM', svm_clf),
                   ('Bagging DT', bag_clf),
                   ('DecisionForest', rfc_clf)]
sclf = StackingClassifier(estimators=base_estimators,
                          final_estimator=logr_clf,
                          cv=2).fit(X_train, y_train)
# sclf.fit(X_train, y_train)
print('Accuracy =', sclf.score(X_train, y_train))

Accuracy = 0.849


Выполните предсказание для изображений, указанных ниже. Определите вероятность отнесения изображений к классу 1 с помощью обученного метаалгоритма. Ответы округлите до тысячных.

In [13]:
# from google.colab.patches import cv2_imshow
list_imgs = ['/content/drive/My Drive/4_ensemble/test/cat.1046.jpg',
             '/content/drive/My Drive/4_ensemble/test/dog.1005.jpg',
             '/content/drive/My Drive/4_ensemble/test/dog.1019.jpg',
             '/content/drive/My Drive/4_ensemble/test/cat.1001.jpg']
X_test = []
for i in list_imgs:
  X_test.append(extract_histogram(cv2.imread(i)))
  # cv2_imshow(cv2.imread(i))

for i in sclf.predict_proba(X_test):
  print(round(i[1], 3))

0.426
0.096
0.271
0.266
